### Importing Dependencies

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

Install:
<ul>
    <li>Transformers</li>
    <li>BeautifulSoup</li>
    <li>Pillow</li>
    <li>Requests</li>
    <li>Pytorch</li>
</ul>


#### Loading the Tokenizer and the `nlptown/bert-base-multilingual-uncased-sentiment` model from HuggingFace

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\hp\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [48]:
# Converting a sentence to tokens
# return_tensors = 'pt' because we are working with PyTorch
tokens = tokenizer.encode('Meh it was good, could be better', return_tensors = 'pt')
tokens

tensor([[  101, 10525, 10243, 10197, 10140, 12050,   117, 12296, 10346, 16197,
           102]])

In [49]:
# Decoding the tokens to form a sentence
# CLS -> Start of the sentence and, SEP -> End of the sentence
tokenizer.decode(tokens[0])

'[CLS] meh it was good, could be better [SEP]'

In [50]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-0.6054,  1.9678,  2.7688, -0.4658, -3.0271]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [51]:
result.logits

tensor([[-0.6054,  1.9678,  2.7688, -0.4658, -3.0271]],
       grad_fn=<AddmmBackward0>)

In [52]:
torch.argmax(result.logits)

tensor(2)

Since the index starts at `0`, but the sentiment categories in this model range from 1 to 5 (1 meaning very bad and 5 meaning good), you need to add `1` to the result to map the indices properly to the sentiment categories.
Instead of getting a zero rating which is not possible you get 1.

In [53]:
int(torch.argmax(result.logits))+1

3

### Scrapping the Yelp website for Restaruant Reviews

In [12]:
request_url = requests.get('https://www.yelp.com/biz/sotto-mare-san-francisco-4?osq=Restaurants')
soup = BeautifulSoup(request_url.text, 'html.parser')
regex = re.compile('.*comment.*')
outputs = soup.find_all('p', {'class': regex})
reviews = [output.text for output in outputs]

In [13]:
reviews

["Really good ! It is worth it waiting a line !And we went to 4:30 but it is start 5:00 pm Ant that time many people waited int front of the door. If you go to 5 pm you don't need to waiting",
 "Excellent!  We had a great time and a great experience.  Made last minute reservations (so thankful there was an opening!) for a party of four.  The service was prompt, the food was tasty (and not salty!), and the ambience was festive.  We ordered the cioppino, salads, scallops, clam chowder, and a pasta dish ... all were so good.  Bottle of chardonnay and space dust on draft to accompany.  Will come back at next opportunity!  One suggestion:  How 'bout a military discount:)!",
 "Reserved here for a party of 4 birthday dinner and it was absolutely excellent seafood with Italian style. We were seated below and it was comfortably out of the cold. Service was attentive and who doesn't love some bread appetizer! Best Crab Cioppono: terrific/5, full of shrimp, clam, octopus, mini shrimp, a little pe

In [15]:
# Converting the scrapped reviews into Data Frame
df = pd.DataFrame(np.array(reviews), columns = ['review'])

In [16]:
df.head()

,review
0,Really good ! It is worth it waiting a line !A...
1,Excellent! We had a great time and a great ex...
2,Reserved here for a party of 4 birthday dinner...
3,I have actually left a 5 star review in the pa...
4,Memorably delicious. We understood the assign...


In [37]:
# Function to determine the sentiment score
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+ 1

In [39]:
# We use 512 because we can only have 512 tokens passed 
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [40]:
df

,review,sentiment
0,Really good ! It is worth it waiting a line !A...,3
1,Excellent! We had a great time and a great ex...,4
2,Reserved here for a party of 4 birthday dinner...,4
3,I have actually left a 5 star review in the pa...,1
4,Memorably delicious. We understood the assign...,4
5,This popular eatery is located in little Italy...,3
6,Food seems to taste better with your loved one...,1
7,Sotto Mare is one of many delicious seafood re...,3
8,"Excellent prices, food quality and service! I ...",4
9,5 STARS.Sotto Mare is easily one of the best p...,4
